In [1]:
import pandas as pd 
import csv 
import numpy as np
import itertools
from fuzzywuzzy import fuzz, process

In [2]:
df = pd.read_csv('/Users/szechan/Documents/a/co_op/cps_webscrape.csv')

In [3]:
df.head()

,Unnamed: 0,raw_data,units,crn,course,program_name
0,6,Download Catalog PDF Files,NaN,NaN,NaN,"Homeland Security, MA"
1,7,"Homeland Security, MA",NaN,NaN,NaN,"Homeland Security, MA"
2,8,Required Courses,NaN,NaN,NaN,"Homeland Security, MA"
3,9,Code Title Hours,NaN,NaN,NaN,"Homeland Security, MA"
4,10,CMN 6050Crisis Communication3,3.0,CMN 6050,Crisis Communication,"Homeland Security, MA"


In [4]:
# How many Master's program are there under CPS? 
df['program_name'].nunique()

25

In [5]:
#https://stackoverflow.com/questions/41476150/removing-space-from-dataframe-columns-in-pandas
df.program_name = df.program_name.str.replace('^ +| +$', '_')

In [6]:
df.program_name = df.program_name.str.replace(',', ' ')

In [7]:
df.program_name = df.program_name.str.replace(' ', '_')

In [8]:
df.program_name.unique()

array(['Homeland_Security__MA', 'Strategic_Intelligence_and_Analysis__MA',
       'Teaching__Elementary_Licensure__MAT',
       'Teaching__Secondary_Licensure__MAT', 'Education__MEd',
       'Higher_Education_Administration__MEd', 'Analytics__MPS',
       'Applied_Machine_Intelligence__MPS', 'Digital_Media__MPS',
       'Digital_Media__MPS—Connect', 'Enterprise_Intelligence__MPS',
       'Geospatial_Services__MPS', 'Informatics__MPS',
       'Applied_Nutrition__MS', 'Commerce_and_Economic_Development__MS',
       'Corporate_and_Organizational_Communication__MS',
       'Criminal_Justice__MS',
       'Global_Studies_and_International_Relations__MS',
       'Human_Resources_Management__MS', 'Leadership__MS',
       'Nonprofit_Management__MS', 'Project_Management__MS',
       'Regulatory_Affairs__MS', 'Respiratory_Care_Leadership__MS',
       'Sports_Leadership__MSLD'], dtype=object)

In [9]:
#https://stackoverflow.com/questions/59304168/python-split-dataframe-into-multiple-dataframes-by-using-for-loop-with-locals-f
for name in df['program_name'].unique():
    locals()['df_' + name] = df[(df.program_name == name)]

In [10]:
#show me a list of dataframe created 
%who_ls DataFrame 

['df',
 'df_Analytics__MPS',
 'df_Applied_Machine_Intelligence__MPS',
 'df_Applied_Nutrition__MS',
 'df_Commerce_and_Economic_Development__MS',
 'df_Corporate_and_Organizational_Communication__MS',
 'df_Criminal_Justice__MS',
 'df_Digital_Media__MPS',
 'df_Digital_Media__MPS—Connect',
 'df_Education__MEd',
 'df_Enterprise_Intelligence__MPS',
 'df_Geospatial_Services__MPS',
 'df_Global_Studies_and_International_Relations__MS',
 'df_Higher_Education_Administration__MEd',
 'df_Homeland_Security__MA',
 'df_Human_Resources_Management__MS',
 'df_Informatics__MPS',
 'df_Leadership__MS',
 'df_Nonprofit_Management__MS',
 'df_Project_Management__MS',
 'df_Regulatory_Affairs__MS',
 'df_Respiratory_Care_Leadership__MS',
 'df_Sports_Leadership__MSLD',
 'df_Strategic_Intelligence_and_Analysis__MA',
 'df_Teaching__Elementary_Licensure__MAT',
 'df_Teaching__Secondary_Licensure__MAT']

In [11]:
df_Teaching__Elementary_Licensure__MAT

,Unnamed: 0,raw_data,units,crn,course,program_name
496,502,Download Catalog PDF Files,NaN,NaN,NaN,Teaching__Elementary_Licensure__MAT
497,503,"Teaching, Elementary Licensure, MAT",NaN,NaN,NaN,Teaching__Elementary_Licensure__MAT
498,504,1The MAT (grades 1–6) has been approved at the...,NaN,NaN,NaN,Teaching__Elementary_Licensure__MAT
499,505,Required Courses,NaN,NaN,NaN,Teaching__Elementary_Licensure__MAT
500,506,Code Title Hours,NaN,NaN,NaN,Teaching__Elementary_Licensure__MAT
...,...,...,...,...,...,...
683,689,EDU 6300Introduction to Language and Linguistics4,4.0,EDU 6300,Introduction to Language and Linguistics,Teaching__Elementary_Licensure__MAT
684,690,EDU 6516Sheltered English Instruction and Asse...,4.0,EDU 6516,Sheltered English Instruction and Assessment,Teaching__Elementary_Licensure__MAT
685,691,EDU 6517Foundations of Teaching English as a S...,4.0,EDU 6517,Foundations of Teaching English as a Second La...,Teaching__Elementary_Licensure__MAT
686,692,EDU 6310Literacy Development and the Academic ...,4.0,EDU 6310,Literacy Development and the Academic Domains,Teaching__Elementary_Licensure__MAT


In [12]:
#remove the trailing and tailing space on the 'course' column
df.course = df.course.str.replace('^ +| +$', '_')

In [13]:
#I want to see if there are any course overlap between to program 1) Informatics 2) PJM 
#create a new dataframe called course_informatics 
course_informatics = df_Informatics__MPS['course']

In [14]:
#allow me to view up to 100 rows 
pd.options.display.max_rows = 100

In [15]:
course_informatics.head(10)

3084                                               NaN
3085                                               NaN
3086                                               NaN
3087                                               NaN
3088                        Foundations of Informatics
3089                       Database Management Systems
3090    Information Technology Strategy and Governance
3091        Information Systems Design and Development
3092         Information Technology Project Management
3093                                               NaN
Name: course, dtype: object

In [16]:
#create a new dataframe for pjm 
course_pjm = df_Project_Management__MS['course']

In [17]:
course_informatics.size

384

In [18]:
course_pjm.size

432

In [20]:
#create a cartesian product between course_informatics and course_pjm
test_1 = list(itertools.product(course_informatics, course_pjm))

In [21]:
#cartesian product 
len(test_1)

165888

In [22]:
#create a new dataframe df_1 that allows me to compare all courses from infromatics and pjm side-by-side. 
df_1 = pd.DataFrame(test_1, columns=['infromatics', 'pjm'])

In [23]:
df_1.head(10)

,infromatics,pjm
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,Foundations of Project Management
7,NaN,Project Management Practices
8,NaN,NaN
9,NaN,Project Management Practices


In [24]:
#replace all NaN to space 
df_1 = df_1.replace(np.nan, '', regex=True)

In [25]:
df_1['Ratio Score'] = df_1.apply(lambda row: fuzz.partial_ratio(*row), axis=1)

In [26]:
df_1

,infromatics,pjm,Ratio Score
0,,,100
1,,,100
2,,,100
3,,,100
4,,,100
...,...,...,...
165883,,Introduction to Analytics,0
165884,,Foundations of Project Business Analysis,0
165885,,Project Business Analysis: Needs Assessment,0
165886,,Project Business Analysis: Requirements Planni...,0


In [47]:
# show to me results based on the following criteria: 1) Ratio Score must be greater than 90 2) both informatics
# and pjm must not equal to space
result_df_1 = df_1[(df_1['Ratio Score'] > 90)&(df_1['infromatics'] != '')& (df_1['pjm']!= '')]

In [45]:
result_df_1.head(n=100)

,infromatics,pjm,Ratio Score
4336,Informatics Capstone,Capstone,100
4444,Informatics Capstone,Capstone,100
4552,Informatics Capstone,Capstone,100
4660,Informatics Capstone,Capstone,100
6536,Introduction to Analytics,Introduction to Analytics,100
6583,Introduction to Analytics,Introduction to Analytics,100
6644,Introduction to Analytics,Introduction to Analytics,100
6691,Introduction to Analytics,Introduction to Analytics,100
6752,Introduction to Analytics,Introduction to Analytics,100
6799,Introduction to Analytics,Introduction to Analytics,100


In [46]:
#drop the duplicates 
result_df_1.drop_duplicates()

,infromatics,pjm,Ratio Score
4336,Informatics Capstone,Capstone,100
6536,Introduction to Analytics,Introduction to Analytics,100
6969,Probability Theory and Introductory Statistics,Probability Theory and Introductory Statistics,100
7402,Intermediate Analytics,Intermediate Analytics,100
7835,Communication and Visualization for Data Analy...,Communication and Visualization for Data Analy...,100
8268,one of the following:,rom one of the following:,98
8287,one of the following:,one of the following:,100
13452,one of the following: 3-,rom one of the following:,94
13471,one of the following: 3-,one of the following:,100
25927,Project Management Practices,Project Management Practices,100


In [ ]:
#https://gist.github.com/bsweger/e5817488d161f37dcbd2